# Handling WOD netCDF files

The purpose of this notebook is to demonstrate the functionality of `WODbottom`.

The demo is organised to show

- Step 1: Explanation

- Step 2: Explanation

Note that when you submit a pull request, you should `clear all outputs` from your python notebook for a cleaner merge.


In [ ]:
import pathlib
import sys

script_dir = pathlib.Path().parent.absolute()
parent_dir = script_dir.parents[0]
sys.path.append(str(parent_dir))
print(parent_dir)

import xarray as xr
import os
from WODbottom import readers, writers, plotters, tools, utilities
import pandas as pd
import bathyreq


In [ ]:
# Specify the path for writing datafiles
data_path = os.path.join(parent_dir, 'data')

In [ ]:
input_dir = '/Users/eddifying/Dropbox/Data/WB_Atlantic_WOD_CTD/'

file_count = len([name for name in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, name))])
print(f'There are {file_count} files in the input directory.')

# Load one file

In [ ]:
# Find the first file in the input_dir that starts with 'wod'
first_file = next((f for f in os.listdir(input_dir) if f.startswith('wod')), None)

if first_file:
    file_path = os.path.join(input_dir, first_file)
    ds = xr.open_dataset(file_path)
    ds
else:
    print("No file starting with 'wod' found in the input directory.")


In [ ]:
ds

In [ ]:
def process_file(file_name, input_dir):
        if file_name.startswith('wod') and file_name.endswith('.nc'):
            file_path = os.path.join(input_dir, file_name)
            ds = xr.open_dataset(file_path, decode_timedelta=True)

            if 'Salinity' in ds and 'Temperature' in ds:
                lat = ds.lat.item()
                lon = ds.lon.item()
                max_z = ds.z.max().item()
                #                max_z_index = ds.z.where(ds.z == max_z, drop=True).squeeze().item()
                temp_at_max_z = ds.Temperature.where(ds.z == max_z, drop=True).squeeze().item()

                #                temp_at_max_z = ds.Temperature.sel(z=max_z, method='nearest').item()
                salinity_at_max_z = ds.Salinity.where(ds.z == max_z, drop=True).squeeze().item()
                #                salinity_at_max_z = ds.Salinity.sel(z=max_z, method='nearest').item()
                if salinity_at_max_z < 0:
                    salinity_at_max_z = float('nan')
                if 'Bottom_Depth' in ds:
                    bottom_depth = ds.Bottom_Depth.item()
                else:
                    bottom_depth = None
                date = ds.date.item()
                wod_unique_cast = ds.wod_unique_cast.item()

                return {
                    'file_name': file_name,
                    'max_z': max_z,
                    'Temperature_at_max_z': temp_at_max_z,
                    'Salinity_at_max_z': salinity_at_max_z,
                    'Bottom_Depth': bottom_depth,
                    'date': date,
                    'Wod_unique_cast': wod_unique_cast,
                    'lat': lat,
                    'lon': lon
                }
            else:
                return None
        return None

if 0:
    results = []

    for file_name in os.listdir(input_dir):
        if file_name.startswith('wod'):
            result = process_file(file_name, input_dir)
            if result:
                results.append(result)
            if len(results) % 10000 == 0:
                print(f'Processed {len(results)} files...')
                # Save results_df to a pickle file
                # Convert results to a DataFrame for better visualization
                results_df = pd.DataFrame(results)
                print(results_df)
                last_wod_unique_cast = results_df['Wod_unique_cast'].iloc[-1]
                results_df.to_pickle(os.path.join(data_path, f'results_df_{last_wod_unique_cast}.pkl'))
                results = []



    # Convert the DataFrame to an xarray Dataset
    results_ds = results_df.set_index(['date']).to_xarray()

    # Rename the dimension to 'wod_profile'
    results_ds = results_ds.rename_dims({'date': 'wod_profile'})

    # Add lat and lon as coordinates
    results_ds = results_ds.set_coords(['lat', 'lon'])
    # Display the xarray Dataset
    results_ds
    # Save results_ds to a NetCDF file
    results_ds.to_netcdf(os.path.join(data_path, 'results_ds.nc'))

In [ ]:

ds = xr.open_dataset(os.path.join(data_path, 'results_ds.nc'))
ds

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Extract latitude and longitude from the results DataFrame
#lats = results_df['lat']
#lons = results_df['lon']
lats = ds['lat'].values
lons = ds['lon'].values

# Create a figure and axis with a specific projection
fig, ax = plt.subplots(figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})

# Add features to the map
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plot the locations with circles
ax.scatter(lons, lats, color='red', s=50, edgecolor='k', transform=ccrs.PlateCarree())

# Set the extent of the map
ax.set_extent([-80, -50, 20, 50], crs=ccrs.PlateCarree())

# Add gridlines
ax.gridlines(draw_labels=True)

# Add a title
ax.set_title('Locations of WOD Casts')

# Show the plot
plt.show()

In [ ]:
lonlim = (-90, -40)
latlim = (25, 55)
# Example usage
ds_bathy = readers.load_bathymetry(lonlim=lonlim, latlim=latlim)
ds_bathy

# Plot the bathymetry
plotters.plot_bathy_and_argo(ds_bathy)
ds_bathy

In [ ]:
import numpy as np
results_ds = ds
# Define the range for latitude and longitude
lat_range = ds_bathy['LATITUDE'].values
lon_range = ds_bathy['LONGITUDE'].values

# Create a meshgrid
lon_grid, lat_grid = np.meshgrid(lon_range, lat_range)
dlon = np.diff(lon_range)
dlat = np.diff(lat_range)

print(f'Latitude grid shape: {lat_grid.shape}')
print(f'Longitude grid shape: {lon_grid.shape}')

# Find all results_ds.lat within 0.1 of lat_grid and results_ds.lon within 0.1 of lon_grid
lat_matches = np.abs(results_ds.lat.values[:, np.newaxis, np.newaxis] - lat_grid) <= 0.1
lon_matches = np.abs(results_ds.lon.values[:, np.newaxis, np.newaxis] - lon_grid) <= 0.1

# Combine the matches to find the points that satisfy both conditions
matches = lat_matches & lon_matches

print(f'Number of matches: {np.sum(matches)}')

In [ ]:
matches.shape

In [ ]:
i, j = np.where(lat_matches)
print(f'Indices where lat_matches is True: i={i}, j={j}')

In [ ]:
ds

In [ ]:
import matplotlib.pyplot as plt

# Plot max_z against Bottom_Depth
plt.figure(figsize=(8, 6))
plt.scatter(ds['max_z'], ds['Bottom_Depth'], alpha=0.5, edgecolor='k')
plt.xlabel('Max Depth (max_z)')
plt.ylabel('Bottom Depth')
plt.title('Max Depth vs Bottom Depth')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calculate the difference between max_z and Bottom_Depth
depth_difference = ds['max_z'] - ds['Bottom_Depth']

# Plot the time series
plt.figure(figsize=(10, 6))
plt.plot(ds['date'], depth_difference, 'o', label='Max Depth - Bottom Depth', alpha=0.7)
plt.xlabel('Date')
plt.ylabel('Depth Difference (m)')
plt.title('Time Series of Max Depth Minus Bottom Depth')
plt.legend()
plt.grid(True)
plt.show()